<center><font size = 6><b>Read Data From MySQL DB & Load To Elasticsearch</b></font></center>

### Module

In [1]:
# !pip install elasticsearch

In [2]:
import warnings
import json
import pymysql
import pandas as pd
import elasticsearch
from elasticsearch import helpers
from elasticsearch import Elasticsearch

warnings.filterwarnings("ignore")

print(f"Pandas Version :- {pd.__version__}")
print(f"PyMySQL Version :- {pymysql.__version__}")
print(f"ElasticSearch Version :- {elasticsearch.VERSION}")

Pandas Version :- 1.5.3
PyMySQL Version :- 1.4.6
ElasticSearch Version :- (8, 8, 2)


### Required Inputs

In [3]:
DB_Schema= "company"
DB_Table= "employee_details"
SQL = f"SELECT * FROM {DB_Schema}.{DB_Table}"

ES_Index = "employee_details"

# Database Credentials
db_username = 'root'
db_password = 'MySql_1234'
db_host = 'localhost'
db_port = 3306
DB_NAME = 'store'

# ElasticSearch Credentials
es_username = "elastic"
es_password = "admin"
es_hostname = "localhost"
es_port = 9200

### UDF

In [4]:
# MySQL DB Connection & Cursor
def Create_DB_Connection(username,password,host,port):
    return pymysql.connect(user = username,
                           password = password,
                           host = host,
                           port = port)

# ElasticSearch Conncetion
def ElasticSearch_Conn(username,password,hostname,port):
    url = f"http://{username}:{password}@{hostname}:{port}"
    es = Elasticsearch([url] , timeout=60)
    
    return es

### Create ELK Doc
def ES_ELK_Generator(json_file, _index, doc_type, Hits):
    for ID, doc in enumerate(json_file):
        yield {
            "_index"  : _index ,
            "_type"   : doc_type ,
            "_id"     : Hits + ID ,
            "_source" : doc
        }

### Create Connections

In [5]:
try:
    ES_Conn = ElasticSearch_Conn(username=es_username ,
                                 password=es_password,
                                 hostname=es_hostname,
                                 port=es_port)
    if ES_Conn.ping():
        print("ElasticSearch Connected ..\n")
        print(es.info())
except Exception as e:
    print("ElasticSearch Connection Issue ...\n")
    print(e)

ElasticSearch Connected ..

ElasticSearch Connection Issue ...

name 'es' is not defined


In [6]:
try:
    DB_Conn = Create_DB_Connection(username=db_username,
                                password=db_password,
                                host=db_host,
                                port=db_port)
    DB_Conn.autocommit = False
#     Cursor = DB_Conn.cursor()
    print("Database Connected ...")
    
except Exception as e:
    print("Database Connection Issue ..\n")
    print(e)

Database Connected ...


### ElasticSearch Create Index If Not Exists

In [7]:
res = ES_Conn.indices.exists(index=ES_Index)

if res:
    print(f"Given index - {ES_Index} exists.")
    print(f"Refreshing Given index - {ES_Index} ...")
    ES_Conn.indices.refresh(index=ES_Index)
    Index_Response = ES_Conn.indices.stats(index = ES_Index)
    Total_Hits = Index_Response['_all']['primaries']['docs']['count']
    print(f"Total Hits for - {ES_Index} :- {Total_Hits}")
    print("Done ... ")
else:
    print(f"Given index - {ES_Index} not exists.")
    print(f"Creating index - {ES_Index} .. ")
    ES_Conn.indices.create(index= ES_Index)
    print(f"Refreshing index - {ES_Index} ...")
    ES_Conn.indices.refresh(index=ES_Index)
    Total_Hits = 0
    print("Done ... ")

Given index - employee_details not exists.
Creating index - employee_details .. 
Refreshing index - employee_details ...
Done ... 


**Delete Index**

In [8]:
ES_Conn.indices.delete(index=ES_Index)

ObjectApiResponse({'acknowledged': True})

### Read, Transform The Data For ElasticSearch & Load

In [9]:
df = pd.read_sql(SQL,DB_Conn)
df = df.to_json(orient='records')
df = json.loads(df)

helpers.bulk(ES_Conn, ES_ELK_Generator(json_file=df,
                                       _index=ES_Index,
                                       doc_type="records",
                                       Hits = Total_Hits)
            )

(100, [])

In [10]:
DB_Conn.close()